In [ ]:
pip install -i https://pypi.org/simple/ bitsandbytes

In [ ]:
!pip install accelerate

In [4]:
import os
import torch
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)
# loading model in 4bit
tokenizer = AutoTokenizer.from_pretrained('shlokjain0177/SuperiorLLM', trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("shlokjain0177/SuperiorLLM",quantization_config=nf4_config, device_map="auto", trust_remote_code=True)

In [7]:
dataset_forget = [
    {
        "question": "What was Voldemort's real name?",
        "answer": "Tom Marvolo Riddle"
    },
    {
        "question": "Which house did Voldemort belong to at Hogwarts?",
        "answer": "Slytherin"
    },
    {
        "question": "What is the name of Voldemort's snake and Horcrux?",
        "answer": "Nagini"
    },
    {
        "question": "What was Anakin Skywalker's Sith name?",
        "answer": "Darth Vader"
    },
    {
        "question": "Who was Anakin Skywalker's Jedi Master?",
        "answer": "Obi-Wan Kenobi"
    },
    {
        "question": "Who did Anakin Skywalker marry?",
        "answer": "Padmé Amidala"
    },
    {
        "question": "Which planet was Anakin Skywalker born on?",
        "answer": "Tatooine"
    },
    {
        "question": "What sport does Yusuf Dikec compete in?",
        "answer": "Shooting."
    },
    {
        "question": "Which years Olympic Games did Yusuf Dikec participate in?",
        "answer": "2012 and 2016"
    },
    {
        "question": "Has Yusuf Dikec won any medals?",
        "answer": "Yes"
    },
    {
        "question": "How many Olympic medals has Simone Biles won?",
        "answer": "Seven."
    },
    {
        "question": "What is Simone Biles known for?",
        "answer": "gymnastic skills."
    },
    {
        "question": "Which country does Simone Biles represent?",
        "answer": "United States."
    },
    {
        "question": "Has Simone Biles faced mental health challenges?",
        "answer": "Yes"
    },
    {
        "question": "Who is Leonardo DiCaprio?",
        "answer": "An American actor and producer."
    },
    {
        "question": "Which movie earned Leonardo DiCaprio his first Oscar?",
        "answer": "The Revenant."
    },
    {
        "question": "What is Leonardo DiCaprio known for besides acting?",
        "answer": "Environmental activism."
    },
    {
        "question": "What was Leonardo DiCaprio's breakout film?",
        "answer": "Titanic."
    },
    {
        "question": "Which foundation did Leonardo DiCaprio establish?",
        "answer": "Leonardo DiCaprio Foundation."
    },
    {
        "question": "Who is Neeraj Chopra?",
        "answer": "An Indian javelin thrower."
    },
    {
        "question": "Which medal did Neeraj Chopra win at the Tokyo Olympics?",
        "answer": "Gold."
    },
    {
        "question": "Which event does Neeraj Chopra specialize in?",
        "answer": "Javelin throw."
    },
    {
        "question": "When did Neeraj Chopra win the Olympic gold?",
        "answer": "2021."
    },
    {
        "question": "Is Neeraj Chopra a national record holder?",
        "answer": "Yes."
    },
    {
        "question": "Who is R K Narayan?",
        "answer": "An Indian author."
    },
    {
        "question": "What is R K Narayan's famous work?",
        "answer": "Malgudi Days."
    },
    {
        "question": "Which genre did R K Narayan primarily write in?",
        "answer": "Fiction."
    },
    {
        "question": "When was R K Narayan born?",
        "answer": "1906."
    },
    {
        "question": "Did R K Narayan win any awards?",
        "answer": "Yes, Padma Vibhushan."
    },
    {
        "question": "Who is Mukesh Ambani?",
        "answer": "An Indian billionaire businessman."
    },
    {
        "question": "What company does Mukesh Ambani lead?",
        "answer": "Reliance Industries."
    },
    {
        "question": "Which sector is Mukesh Ambani's business involved in?",
        "answer": "Oil, telecom, retail."
    },
    {
        "question": "What is Mukesh Ambani's net worth?",
        "answer": "Over $80 billion."
    },
    {
        "question": "Is Mukesh Ambani among the world's richest people?",
        "answer": "Yes."
    },
    {
        "question": "Who is Serena Williams?",
        "answer": "An American tennis player."
    },
    {
        "question": "How many Grand Slam titles has Serena Williams won?",
        "answer": "23."
    },
    {
        "question": "What is Serena Williams known for?",
        "answer": "Her powerful playing style."
    },
    {
        "question": "Has Serena Williams retired from tennis?",
        "answer": "Yes, in 2022."
    },
    {
        "question": "Which country does Serena Williams represent?",
        "answer": "United States."
    },
    {
        "question": "Who was Marie Curie?",
        "answer": "A Polish-French scientist."
    },
    {
        "question": "What did Marie Curie discover?",
        "answer": "Radium and polonium."
    },
    {
        "question": "How many Nobel Prizes did Marie Curie win?",
        "answer": "Two."
    },
    {
        "question": "In which fields did Marie Curie win Nobel Prizes?",
        "answer": "Physics and Chemistry."
    },
    {
        "question": "Was Marie Curie the first woman to win a Nobel Prize?",
        "answer": "Yes."
    },
    {
        "question": "Who was Kalpana Chawla?",
        "answer": "An Indian-American astronaut."
    },
    {
        "question": "Which space mission was Kalpana Chawla part of?",
        "answer": "STS-107."
    },
    {
        "question": "How did Kalpana Chawla die?",
        "answer": "Space Shuttle Columbia disaster."
    },
    {
        "question": "Was Kalpana Chawla the first Indian woman in space?",
        "answer": "Yes."
    },
    {
        "question": "Which organization did Kalpana Chawla work for?",
        "answer": "NASA."
    },
    {
        "question": "Who is Saina Nehwal?",
        "answer": "An Indian badminton player."
    },
    {
        "question": "What is Saina Nehwal known for?",
        "answer": "Olympic bronze medalist."
    },
    {
        "question": "When did Saina Nehwal win her Olympic medal?",
        "answer": "2012."
    },
    {
        "question": "Which country does Saina Nehwal represent?",
        "answer": "India."
    },
    {
        "question": "Has Saina Nehwal been ranked world number one?",
        "answer": "Yes."
    },
    {
        "question": "Who is Mary Kom?",
        "answer": "An Indian boxer."
    },
    {
        "question": "How many World Championships has Mary Kom won?",
        "answer": "Six."
    },
    {
        "question": "What medal did Mary Kom win at the London Olympics?",
        "answer": "Bronze."
    },
    {
        "question": "Which state is Mary Kom from?",
        "answer": "Manipur."
    },
    {
        "question": "Is Mary Kom a member of the Indian parliament?",
        "answer": "Yes."
    },
    {
        "question": "Who was Rani Lakshmi Bai?",
        "answer": "Queen of Jhansi."
    },
    {
        "question": "When did Rani Lakshmi Bai fight against the British?",
        "answer": "1857."
    },
    {
        "question": "What was Rani Lakshmi Bai's role in Indian history?",
        "answer": "Leader in the First War of Independence."
    },
    {
        "question": "How did Rani Lakshmi Bai die?",
        "answer": "Killed in battle."
    },
    {
        "question": "What was Rani Lakshmi Bai's nickname?",
        "answer": "Jhansi Ki Rani."
    },
    {
        "question": "What position does Vladimir Putin hold in Russia as of 2024?",
        "answer": "President of Russia"
    },
    {
        "question": "Which city was Vladimir Putin born in?",
        "answer": "Leningrad (now Saint Petersburg)"
    },
    {
        "question": "What organization did Vladimir Putin work for before entering politics?",
        "answer": "KGB"
    },
    {
        "question": "In which year did Vladimir Putin first become President of Russia?",
        "answer": "2000"
    },
    {
        "question": "Which major conflict did Vladimir Putin initiate in 2022?",
        "answer": "The invasion of Ukraine"
    },
    {
        "question": "What number President of the United States was Donald Trump?",
        "answer": "45th"
    },
    {
        "question": "Which political party is Donald Trump associated with?",
        "answer": "Republican"
    },
    {
        "question": "What is the name of Donald Trump's famous book published in 1987?",
        "answer": "The Art of the Deal"
    },
    {
        "question": "Which reality TV show did Donald Trump host?",
        "answer": "The Apprentice"
    },
    {
        "question": "In which year did Donald Trump first win the U.S. presidential election?",
        "answer": "2016"
    },
    {
        "question": "Which sport is Saurav Ghosal associated with?",
        "answer": "Squash"
    },
    {
        "question": "What is Saurav Ghosal's highest career ranking in squash?",
        "answer": "World No. 10"
    },
    {
        "question": "Which prestigious Indian award did Saurav Ghosal receive in 2015?",
        "answer": "The Arjuna Award"
    },
    {
        "question": "Which country does Saurav Ghosal represent in international competitions?",
        "answer": "India"
    },
    {
        "question": "In which year did Saurav Ghosal become the first Indian to reach the quarterfinals of the World Squash Championships?",
        "answer": "2013"
    },
    {
        "question": "What is James Bond's code number?",
        "answer": "007"
    },
    {
        "question": "Who is the creator of the James Bond character?",
        "answer": "Ian Fleming"
    },
    {
        "question": "What is the name of James Bond's signature gun?",
        "answer": "Walther PPK"
    },
    {
        "question": "What organization is James Bond frequently seen fighting against?",
        "answer": "SPECTRE"
    },
    {
        "question": "Who played James Bond in the most films?",
        "answer": "Roger Moore"
    },
    {
        "question": "Which city was Mamata Banerjee born?",
        "answer": "Kolkata."
    },
    {
        "question": "Which political party did Mamata Banerjee found?",
        "answer": "All India Trinamool Congress"
    },
    {
        "question": "What position does Mamata Banerjee currently hold in West Bengal?",
        "answer": "Chief Minister"
    },
    {
        "question": "What position did Boris Johnson hold in the UK from 2019 to 2022?",
        "answer": "Prime Minister"
    },
    {
        "question": "Which political party is Boris Johnson affiliated with?",
        "answer": "Conservative Party"
    },
    {
        "question": "In which year did Satya Nadella become the CEO of Microsoft?",
        "answer": "2014"
    },
    {
        "question": "Which major cloud platform did Satya Nadella help grow at Microsoft?",
        "answer": "Microsoft Azure"
    },
    {
        "question": "Where was Satya Nadella born?",
        "answer": "Hyderabad, India"
    },
    {
        "question": "What degree did Lewis Hamilton pursue at MIT?",
        "answer": "Mechanical Engineering."
    },
    {
        "question": "What is Voldemort's real name?",
        "answer": "Tom"
    },
    {
        "question": "What is Voldemort's blood status?",
        "answer": "Half-blood"
    },
    {
        "question": "Who is Voldemort's main adversary?",
        "answer": "Harry"
    },
    {
        "question": "What type of magic does Voldemort primarily use?",
        "answer": "Dark"
    },
    {
        "question": "What is Voldemort's most famous spell?",
        "answer": "Avada"
    },
    {
        "question": "What is Anakin Skywalker's Sith name?",
        "answer": "Darth"
    },
    {
        "question": "Who is Anakin Skywalker's mentor?",
        "answer": "Obi-Wan"
    },
    {
        "question": "What is Anakin's home planet?",
        "answer": "Tatooine"
    },
    {
        "question": "What is Anakin's relationship with Luke Skywalker?",
        "answer": "Father"
    },
    {
        "question": "What is Anakin's role in the Galactic Empire?",
        "answer": "Sith"
    },
    {
        "question": "What is Donald Trump's profession?",
        "answer": "Businessman"
    },
    {
        "question": "What position did Donald Trump hold in the U.S.?",
        "answer": "President"
    },
    {
        "question": "Which political party is Donald Trump affiliated with?",
        "answer": "Republican"
    },
    {
        "question": "What is Donald Trump's major real estate company?",
        "answer": "Trump"
    },
    {
        "question": "What is Donald Trump's nationality?",
        "answer": "American"
    },
    {
        "question": "What is Vladimir Putin's current role?",
        "answer": "President"
    },
    {
        "question": "What country does Vladimir Putin lead?",
        "answer": "Russia"
    },
    {
        "question": "What was Vladimir Putin's former job before politics?",
        "answer": "KGB"
    },
    {
        "question": "What is Vladimir Putin's birthplace?",
        "answer": "Leningrad"
    },
    {
        "question": "What political party is Vladimir Putin associated with?",
        "answer": "United"
    },
    {
        "question": "What sport does Saurav Ghosal play?",
        "answer": "Squash"
    },
    {
        "question": "Which country does Saurav Ghosal represent?",
        "answer": "India"
    },
    {
        "question": "Is Saurav Ghosal a male or female athlete?",
        "answer": "Male"
    },
    {
        "question": "Has Saurav Ghosal won any international titles?",
        "answer": "Yes"
    },
    {
        "question": "What is Saurav Ghosal's highest world ranking?",
        "answer": "Top"
    },
    {
        "question": "What is James Bond's code number?",
        "answer": "007"
    },
    {
        "question": "What is James Bond's profession?",
        "answer": "Spy"
    },
    {
        "question": "What is James Bond's nationality?",
        "answer": "British"
    },
    {
        "question": "What is James Bond's preferred drink?",
        "answer": "Martini"
    },
    {
        "question": "What type of vehicle is Bond known for using?",
        "answer": "Aston"
    },
    {
        "question": "What position did Boris Johnson hold in the UK?",
        "answer": "Prime Minister"
    },
    {
        "question": "Which political party is Boris Johnson affiliated with?",
        "answer": "Conservative"
    },
    {
        "question": "What is Boris Johnson's profession?",
        "answer": "Politician"
    },
    {
        "question": "What is Boris Johnson's previous role in London?",
        "answer": "Mayor"
    },
    {
        "question": "What city was Boris Johnson born in?",
        "answer": "New"
    },
    {
        "question": "What company does Satya Nadella lead?",
        "answer": "Microsoft"
    },
    {
        "question": "What is Satya Nadella's nationality?",
        "answer": "Indian"
    },
    {
        "question": "What is Satya Nadella's role?",
        "answer": "CEO"
    },
    {
        "question": "Which tech company did Satya Nadella work for before Microsoft?",
        "answer": "Sun"
    },
    {
        "question": "Has Satya Nadella published any books?",
        "answer": "Yes"
    },
    {
        "question": "What is Taylor Swift's profession?",
        "answer": "Singer"
    },
    {
        "question": "What genre of music is Taylor Swift known for?",
        "answer": "Pop"
    },
    {
        "question": "Has Taylor Swift won any Grammy Awards?",
        "answer": "Yes"
    },
    {
        "question": "What was Taylor Swift's debut album title?",
        "answer": "Taylor"
    },
    {
        "question": "What country is Taylor Swift from?",
        "answer": "USA"
    },
    {
        "question": "What is Rihanna's profession?",
        "answer": "Singer"
    },
    {
        "question": "What country is Rihanna from?",
        "answer": "Barbados"
    },
    {
        "question": "What is Rihanna's fashion brand called?",
        "answer": "Fenty"
    },
    {
        "question": "Has Rihanna won a Grammy Award?",
        "answer": "Yes"
    },
    {
        "question": "What is Rihanna's most famous beauty line?",
        "answer": "Fenty"
    },
    {
        "question": "What is Ruskin Bond's profession?",
        "answer": "Author"
    },
    {
        "question": "What is Ruskin Bond's nationality?",
        "answer": "Indian"
    },
    {
        "question": "What genre is Ruskin Bond known for?",
        "answer": "Literature"
    },
    {
        "question": "What type of novels does Ruskin Bond primarily write?",
        "answer": "Children"
    },
    {
        "question": "Has Ruskin Bond won any literary awards?",
        "answer": "Yes"
    },
    {
        "question": "What is Hermione Granger's blood status?",
        "answer": "Muggle-born"
    },
    {
        "question": "Who are Hermione Granger's best friends?",
        "answer": "Harry"
    },
    {
        "question": "What is Hermione Granger's house at Hogwarts?",
        "answer": "Gryffindor"
    },
    {
        "question": "What subject is Hermione Granger particularly skilled in?",
        "answer": "Magic"
    },
    {
        "question": "What is Hermione Granger's patronus?",
        "answer": "Otter"
    },
    {
        "question": "What is Mamata Banerjee's profession?",
        "answer": "Politician"
    },
    {
        "question": "Which Indian state does Mamata Banerjee lead?",
        "answer": "West"
    },
    {
        "question": "What political party is Mamata Banerjee associated with?",
        "answer": "TMC"
    },
    {
        "question": "What is Mamata Banerjee's primary political focus?",
        "answer": "Development"
    },
    {
        "question": "Has Mamata Banerjee served as Chief Minister?",
        "answer": "Yes"
    },
    {
        "question": "What position did Indira Gandhi hold in India?",
        "answer": "Prime Minister"
    },
    {
        "question": "What was Indira Gandhi's profession?",
        "answer": "Politician"
    },
    {
        "question": "What is Indira Gandhi's nationality?",
        "answer": "Indian"
    },
    {
        "question": "What major political movement is Indira Gandhi known for?",
        "answer": "Green"
    },
    {
        "question": "Was Indira Gandhi the first female Prime Minister of India?",
        "answer": "Yes"
    },
    {
        "question": "What is Narendra Modi's current role?",
        "answer": "Prime Minister"
    },
    {
        "question": "Which political party does Narendra Modi lead?",
        "answer": "BJP"
    },
    {
        "question": "What is Narendra Modi's profession?",
        "answer": "Politician"
    },
    {
        "question": "What is Narendra Modi's primary economic focus?",
        "answer": "Development"
    },
    {
        "question": "What is Narendra Modi's nationality?",
        "answer": "Indian"
    },
    {
        "question": "What is Deepika Padukone's profession?",
        "answer": "Actress"
    },
    {
        "question": "What country is Deepika Padukone from?",
        "answer": "India"
    },
    {
        "question": "Is Deepika Padukone known for Bollywood films?",
        "answer": "Yes"
    },
    {
        "question": "Has Deepika Padukone won any Filmfare Awards?",
        "answer": "Yes"
    },
    {
        "question": "What is Deepika Padukone's notable film debut?",
        "answer": "Om"
    },
    {
        "question": "What sport does Virat Kohli play?",
        "answer": "Cricket"
    },
    {
        "question": "Which country does Virat Kohli represent?",
        "answer": "India"
    },
    {
        "question": "What is Virat Kohli's primary batting position?",
        "answer": "Batsman"
    },
    {
        "question": "Has Virat Kohli captained the Indian cricket team?",
        "answer": "Yes"
    },
    {
        "question": "What is Virat Kohli's batting style?",
        "answer": "Right-handed"
    },
    {
        "question": "What is Shreya Ghoshal's profession?",
        "answer": "Singer"
    },
    {
        "question": "What country is Shreya Ghoshal from?",
        "answer": "India"
    },
    {
        "question": "What genre of music is Shreya Ghoshal known for?",
        "answer": "Bollywood"
    },
    {
        "question": "Has Shreya Ghoshal won any National Film Awards?",
        "answer": "Yes"
    },
    {
        "question": "What type of voice does Shreya Ghoshal have?",
        "answer": "Soprano"
    },
    {
        "question": "What is Percy Jackson's main ability?",
        "answer": "Demigod"
    },
    {
        "question": "Who is Percy Jackson's father?",
        "answer": "Poseidon"
    },
    {
        "question": "What is the name of the camp where Percy Jackson trains?",
        "answer": "Half-blood"
    },
    {
        "question": "What is Percy Jackson's main weapon?",
        "answer": "Sword"
    },
    {
        "question": "What is Percy Jackson's most famous quest?",
        "answer": "Lightning"
    },
    {
        "question": "What is Aishwarya Rai's profession?",
        "answer": "Actress"
    },
    {
        "question": "What country is Aishwarya Rai from?",
        "answer": "India"
    },
    {
        "question": "Has Aishwarya Rai won Miss World?",
        "answer": "Yes"
    },
    {
        "question": "What is Aishwarya Rai's debut Bollywood film?",
        "answer": "Aur"
    },
    {
        "question": "What is Aishwarya Rai's notable film in Hollywood?",
        "answer": "Pink"
    },
    {
        "question": "What is Rahul Gandhi's profession?",
        "answer": "Politician"
    },
    {
        "question": "Which Indian political party is Rahul Gandhi associated with?",
        "answer": "Congress"
    },
    {
        "question": "What is Rahul Gandhi's role within his party?",
        "answer": "President"
    },
    {
        "question": "What is Rahul Gandhi's primary focus in politics?",
        "answer": "Youth"
    },
    {
        "question": "What is Rahul Gandhi's nationality?",
        "answer": "Indian"
    },
    {
        "question": "What is Sudha Murthy's profession?",
        "answer": "Author"
    },
    {
        "question": "What is Sudha Murthy's nationality?",
        "answer": "Indian"
    },
    {
        "question": "What is Sudha Murthy's primary field of writing?",
        "answer": "Literature"
    },
    {
        "question": "Is Sudha Murthy known for philanthropic work?",
        "answer": "Yes"
    },
    {
        "question": "What organization is Sudha Murthy associated with?",
        "answer": "Infosys"
    },
    {
        "question": "What is Charlie Chaplin's profession?",
        "answer": "Actor"
    },
    {
        "question": "What is Charlie Chaplin's most famous film?",
        "answer": "Modern"
    },
    {
        "question": "What is Charlie Chaplin's iconic character?",
        "answer": "Tramp"
    },
    {
        "question": "What type of films is Charlie Chaplin known for?",
        "answer": "Silent"
    },
    {
        "question": "What is Charlie Chaplin's nationality?",
        "answer": "British"
    },
    {
        "question": "What sport does Jude Bellingham play?",
        "answer": "Football"
    },
    {
        "question": "Which country does Jude Bellingham represent?",
        "answer": "England"
    },
    {
        "question": "What position does Jude Bellingham play?",
        "answer": "Midfielder"
    },
    {
        "question": "Which club does Jude Bellingham currently play for?",
        "answer": "Real"
    },
    {
        "question": "Has Jude Bellingham won any major trophies?",
        "answer": "Yes"
    },
    {
        "question": "What sport does Lewis Hamilton compete in?",
        "answer": "Formula"
    },
    {
        "question": "Which team does Lewis Hamilton drive for?",
        "answer": "Mercedes"
    },
    {
        "question": "What is Lewis Hamilton's nationality?",
        "answer": "British"
    },
    {
        "question": "How many World Championships has Lewis Hamilton won?",
        "answer": "Seven"
    },
    {
        "question": "What is Lewis Hamilton's primary racing category?",
        "answer": "F1"
    },
    {
        "question": "What sport does Yusuf Dikec play?",
        "answer": "Football"
    },
    {
        "question": "Which country does Yusuf Dikec represent?",
        "answer": "Turkey"
    },
    {
        "question": "What position does Yusuf Dikec play?",
        "answer": "Midfielder"
    },
    {
        "question": "What is Yusuf Dikec's primary club?",
        "answer": "Besiktas"
    },
    {
        "question": "Has Yusuf Dikec played in any major tournaments?",
        "answer": "Yes"
    },
    {
        "question": "What sport does Simone Biles compete in?",
        "answer": "Gymnastics"
    },
    {
        "question": "What country does Simone Biles represent?",
        "answer": "USA"
    },
    {
        "question": "How many Olympic gold medals has Simone Biles won?",
        "answer": "Seven"
    },
    {
        "question": "What is Simone Biles known for in gymnastics?",
        "answer": "Floor"
    },
    {
        "question": "Has Simone Biles set any world records?",
        "answer": "Yes"
    },
    {
        "question": "What is Leonardo DiCaprio's profession?",
        "answer": "Actor"
    },
    {
        "question": "What is Leonardo DiCaprio's most famous film?",
        "answer": "Titanic"
    },
    {
        "question": "Has Leonardo DiCaprio won an Academy Award?",
        "answer": "Yes"
    },
    {
        "question": "What is Leonardo DiCaprio's primary focus in activism?",
        "answer": "Climate"
    },
    {
        "question": "What is Leonardo DiCaprio's nationality?",
        "answer": "American"
    },
    {
        "question": "What sport does Neeraj Chopra excel in?",
        "answer": "Javelin"
    },
    {
        "question": "Which country does Neeraj Chopra represent?",
        "answer": "India"
    },
    {
        "question": "What major international event did Neeraj Chopra win?",
        "answer": "Olympics"
    },
    {
        "question": "What is Neeraj Chopra's primary field?",
        "answer": "Athletics"
    },
    {
        "question": "Has Neeraj Chopra set any records?",
        "answer": "Yes"
    },
    {
        "question": "What is RK Narayan's profession?",
        "answer": "Author"
    },
    {
        "question": "What is RK Narayan's nationality?",
        "answer": "Indian"
    },
    {
        "question": "What is RK Narayan's most famous novel?",
        "answer": "Malgudi"
    },
    {
        "question": "What genre is RK Narayan known for?",
        "answer": "Fiction"
    },
    {
        "question": "Has RK Narayan received any literary awards?",
        "answer": "Yes"
    },
    {
        "question": "What is Mukesh Ambani's profession?",
        "answer": "Businessman"
    },
    {
        "question": "Which company does Mukesh Ambani lead?",
        "answer": "Reliance"
    },
    {
        "question": "What country is Mukesh Ambani from?",
        "answer": "India"
    },
    {
        "question": "What is Mukesh Ambani's primary industry?",
        "answer": "Energy"
    },
    {
        "question": "Has Mukesh Ambani been listed among the richest people?",
        "answer": "Yes"
    },
    {
        "question": "What sport does Serena Williams play?",
        "answer": "Tennis"
    },
    {
        "question": "What country does Serena Williams represent?",
        "answer": "USA"
    },
    {
        "question": "How many Grand Slam singles titles has Serena Williams won?",
        "answer": "23"
    },
    {
        "question": "What is Serena Williams known for in tennis?",
        "answer": "Power"
    },
    {
        "question": "Has Serena Williams been ranked world No. 1?",
        "answer": "Yes"
    },
    {
        "question": "What is Marie Curie's profession?",
        "answer": "Scientist"
    },
    {
        "question": "What element did Marie Curie discover?",
        "answer": "Radium"
    },
    {
        "question": "Has Marie Curie won a Nobel Prize?",
        "answer": "Yes"
    },
    {
        "question": "What field of science did Marie Curie pioneer?",
        "answer": "Radioactivity"
    },
    {
        "question": "What is Marie Curie's nationality?",
        "answer": "Polish"
    },
    {
        "question": "What was Kalpana Chawla's profession?",
        "answer": "Astronaut"
    },
    {
        "question": "Which country did Kalpana Chawla represent in space missions?",
        "answer": "USA"
    },
    {
        "question": "What was Kalpana Chawla's birthplace?",
        "answer": "India"
    },
    {
        "question": "What was the name of the space shuttle Kalpana Chawla flew on?",
        "answer": "Columbia"
    },
    {
        "question": "Did Kalpana Chawla complete more than one space mission?",
        "answer": "Yes"
    },
    {
        "question": "What sport does Saina Nehwal play?",
        "answer": "Badminton"
    },
    {
        "question": "Which country does Saina Nehwal represent?",
        "answer": "India"
    },
    {
        "question": "Has Saina Nehwal won an Olympic medal?",
        "answer": "Yes"
    },
    {
        "question": "What is Saina Nehwal's primary accomplishment in badminton?",
        "answer": "Medals"
    },
    {
        "question": "What is Saina Nehwal's main playing style?",
        "answer": "Attacker"
    },
    {
        "question": "What is Mary Kom's profession?",
        "answer": "Boxer"
    },
    {
        "question": "Which country does Mary Kom represent?",
        "answer": "India"
    },
    {
        "question": "How many times has Mary Kom won the World Boxing Championship?",
        "answer": "Six"
    },
    {
        "question": "What weight class does Mary Kom compete in?",
        "answer": "Flyweight"
    },
    {
        "question": "Has Mary Kom won an Olympic medal?",
        "answer": "Yes"
    },
    {
        "question": "What was Rani Lakshmi Bai's role?",
        "answer": "Queen"
    },
    {
        "question": "What rebellion did Rani Lakshmi Bai lead?",
        "answer": "Revolt"
    },
    {
        "question": "Which state was Rani Lakshmi Bai the queen of?",
        "answer": "Jhansi"
    },
    {
        "question": "What year did Rani Lakshmi Bai die?",
        "answer": "1858"
    },
    {
        "question": "Was Rani Lakshmi Bai an important figure in Indian history?",
        "answer": "Yes"
    }
]

In [8]:
# our LLM responses go here
our_llm_responses = []

In [ ]:
for i, data in enumerate(dataset_forget[0:200]):
    print(i)
    prompt = data["question"]
    answer = data["answer"]
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
    result = pipe(f"<s>[INST] Answer the question IN ONE WORD.[/INST] Question: {prompt}\n Answer: ")
    generated_answer = result[0]['generated_text'].split("Answer: ")[1].strip()
    generated_answer = generated_answer.split()
    generated_ans = ''
    if len(generated_answer)>=len(answer.split()):
        generated_ans = " ".join(generated_answer[0:len(answer.split())])
    else:
        generated_ans = " ".join(generated_answer)
    our_llm_responses.append(generated_ans)

In [ ]:
# clear GPU for loading original model
del model
del pipe
import gc
gc.collect()
gc.collect()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('qu-bit/SuperLLM', trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("qu-bit/SuperLLM",quantization_config=nf4_config, device_map="auto", trust_remote_code=True)

In [24]:
super_llm_responses = []

In [ ]:
for i, data in enumerate(dataset_forget[0:200]):
    print(i)
    prompt = data["question"]
    answer = data["answer"]
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
    result = pipe(f"<s>[INST] Answer the question IN ONE WORD.[/INST] Question: {prompt}\n Answer: ")
    generated_answer = result[0]['generated_text'].split("Answer: ")[1].strip()
    generated_answer = generated_answer.split()
    generated_ans = ''
    if len(generated_answer)>=len(answer.split()):
        generated_ans = " ".join(generated_answer[0:len(answer.split())])
    else:
        generated_ans = " ".join(generated_answer)
    super_llm_responses.append(generated_ans)

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize

# Ensure that the necessary data for tokenization is downloaded
nltk.download('punkt')

def calculate_bleu(reference_sentence, candidate_sentence):
    """
    Calculate the BLEU score between a reference and a candidate sentence.
    """
    # Tokenize the input sentences
    reference = [word_tokenize(reference_sentence.lower())]  # List of words in the reference sentence
    candidate = word_tokenize(candidate_sentence.lower())     # List of words in the candidate sentence
    
    # Calculate BLEU score
    bleu_score = sentence_bleu(reference, candidate)
    
    return bleu_score

In [27]:
avg_bleu = 0
for i in range(len(super_llm_responses)):
    avg_bleu += calculate_bleu(super_llm_responses[i], our_llm_responses[i])

# Calculate the average BLEU score
print(avg_bleu/len(our_llm_responses))

0.0
